In [ ]:
!pip install pandas openpyxl

In [12]:
import pandas as pd
import sqlite3
import os

database_path = 'fge_masterproeven.db'
excel_path = 'Data dump MO in masterproeven 22 23 221123.xlsx'

# Excel inlezen, whitespace in kolmnamen wegsmurfen 
excel_data = pd.read_excel(excel_path, engine='openpyxl')
excel_data.columns = excel_data.columns.str.strip()

# sqlite database maken
if os.path.exists(database_path):
    os.remove(database_path)
conn = sqlite3.connect('fge_masterproeven.db')
cursor = conn.cursor()

# tabellen aanmaken (redelijk dicht bij Django hoop ik)
cursor.execute('''CREATE TABLE IF NOT EXISTS promotor (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT UNIQUE NOT NULL)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS category (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT UNIQUE NOT NULL)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS keyword (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    word TEXT UNIQUE NOT NULL)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS student (
                    student_id TEXT PRIMARY KEY,
                    name TEXT NOT NULL)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS thesis (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    promotor_id INTEGER,
                    category_id INTEGER,
                    academic_year TEXT,
                    title TEXT,
                    summary TEXT,
                    FOREIGN KEY(promotor_id) REFERENCES promotor(id),
                    FOREIGN KEY(category_id) REFERENCES category(id))''')

cursor.execute('''CREATE TABLE IF NOT EXISTS student_thesis (
                    student_id TEXT NOT NULL,
                    thesis_id INTEGER NOT NULL,
                    PRIMARY KEY(student_id, thesis_id),
                    FOREIGN KEY(student_id) REFERENCES student(student_id),
                    FOREIGN KEY(thesis_id) REFERENCES thesis(id))''')

cursor.execute('''CREATE TABLE IF NOT EXISTS thesis_keywords (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    thesis_id INTEGER NOT NULL,
                    keyword_id INTEGER NOT NULL,
                    FOREIGN KEY(thesis_id) REFERENCES thesis(id),
                    FOREIGN KEY(keyword_id) REFERENCES keyword(id))''')

# een thesis kan door meer dan één student ingediend worden, dict aanmaken om unieke thesissen bij te houden
thesis_dict = {}

for index, row in excel_data.iterrows():
    # Promotor 
    cursor.execute("INSERT OR IGNORE INTO promotor (name) VALUES (?)", (row['promotor'],))
    conn.commit()  # Commit om de IGNORE INSERT te bewaren
    cursor.execute("SELECT id FROM promotor WHERE name=?", (row['promotor'],))
    promotor = cursor.fetchone()
    promotor_id = promotor[0] if promotor else None

    # Categorie
    cursor.execute("INSERT OR IGNORE INTO category (name) VALUES (?)", (row['Categorie'],))
    conn.commit()  # Commit om de IGNORE INSERT te bewaren
    cursor.execute("SELECT id FROM category WHERE name=?", (row['Categorie'],))
    category = cursor.fetchone()
    category_id = category[0] if category else None

    # Student 
    cursor.execute("INSERT OR IGNORE INTO student (student_id, name) VALUES (?, ?)",
                   (row['student id'], row['naam student']))

    # unieke key maken voor masterproef (NB deze velden lijken altijd consistent ingevuld te zijn, de summary bv. niet altijd :/)
    thesis_key = (row['promotor'], row['Categorie'], row['Academiejaar'])

    if thesis_key not in thesis_dict:
        # masterproef toevoegen als niet bestaand
        cursor.execute("INSERT INTO thesis (promotor_id, category_id, academic_year, title, summary) VALUES (?, ?, ?, ?, ?)",
                       (promotor_id, category_id, row['Academiejaar'], row['Titel'], row['Samenvatting']))
        thesis_id = cursor.lastrowid
        thesis_dict[thesis_key] = thesis_id
    else:
        thesis_id = thesis_dict[thesis_key]

    # Link Student en Thesis
    cursor.execute("INSERT OR IGNORE INTO student_thesis (student_id, thesis_id) VALUES (?, ?)",
                   (row['student id'], thesis_id))

    # keywords splitten en aanmaken
    keywords = str(row['Kernwoorden']).split(';')
    for word in keywords:
        word = word.strip()
        if word:
            cursor.execute(
                "INSERT OR IGNORE INTO keyword (word) VALUES (?)", (word,))
            cursor.execute("SELECT id FROM keyword WHERE word=?", (word,))
            keyword_id = cursor.fetchone()[0]

            # masterproef en keywords linken
            cursor.execute(
                "INSERT OR IGNORE INTO thesis_keywords (thesis_id, keyword_id) VALUES (?, ?)", (thesis_id, keyword_id))

# klaar!
conn.commit()
conn.close()